In [1]:
import os
import re
import json
import pickle 
from pathlib import Path
import yaml

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import scipy
import skimage
import pandas as pd
import torch
import torchvision
import torchvision.transforms.functional as TF
import tifffile
from sklearn.metrics import adjusted_rand_score
from scipy.cluster.hierarchy import dendrogram
from einops import rearrange, repeat

In [2]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
import mushroom.utils as utils
import mushroom.visualization.utils as vis_utils
import mushroom.data.datasets as datasets
import mushroom.data.visium as visium
import mushroom.data.xenium as xenium
import mushroom.data.multiplex as multiplex
import mushroom.data.he as he
import mushroom.registration.bigwarp as bigwarp
import mushroom.visualization.helpers as helpers
from mushroom.mushroom import Mushroom

In [6]:
project_dir = Path('/data/estorrs/mushroom/data/projects/submission_v1')
output_dir = project_dir / 'analysis' / 'region_characterization'
output_dir.mkdir(parents=True, exist_ok=True)

In [7]:
source_root = '/diskmnt/Projects/Users/estorrs/mushroom/data'
target_root = '/data/estorrs/mushroom/data'

def alter_filesystem(config, source_root, target_root):
    for entry in config['sections']:
        for mapping in entry['data']:
            mapping['filepath'] = mapping['filepath'].replace(source_root, target_root)
    
    if 'trainer_kwargs' in config and config['trainer_kwargs']['data_mask'] is not None:
        config['trainer_kwargs']['data_mask'] = config['trainer_kwargs']['data_mask'].replace(source_root, target_root)
        
    return config

In [8]:
case_order = [
    'HT206B1',
    'HT268B1',
    'HT339B2-H1',
    'HT397B1',
    'HT565B1-H2',
    'HT704B1',
    'HT891Z1',
    'HT913Z1',
    'S18-5591-C8',
    'S18-9906',
]

In [9]:
fps = sorted(utils.listfiles(project_dir, regex=r'mushroom_50res/config.yaml$'))
fps = [fp for fp in fps if fp.split('/')[-3] in case_order]
fps

['/data/estorrs/mushroom/data/projects/submission_v1/HT206B1/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/HT268B1/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/HT339B2-H1/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/HT397B1/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/HT565B1-H2/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/HT704B1/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/HT891Z1/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/HT913Z1/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/S18-5591-C8/mushroom_50res/config.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/S18-9906/mushroom_50res/config.yaml']

In [10]:
case_to_config = {fp.split('/')[-3]:yaml.safe_load(open(fp)) for fp in fps}
case_to_config.keys()

dict_keys(['HT206B1', 'HT268B1', 'HT339B2-H1', 'HT397B1', 'HT565B1-H2', 'HT704B1', 'HT891Z1', 'HT913Z1', 'S18-5591-C8', 'S18-9906'])

In [11]:
multiplex_mapping = {
    'keratin-5': 'ck5',
    'keratin-14': 'ck14',
}
pattern = r'[ -]?\(?[dD]\)?$'
def alter_name(x):
    x = re.sub(pattern, '', x)
    x = x.lower()
    x = x.replace(' ', '-')
    x = multiplex_mapping.get(x, x)
    return x

In [12]:
def generate_labeled_regions(x, cut=10, area=10):
    mask = x > cut
    # mask = skimage.filters.gaussian(x, sigma=.5)
    # mask = mask > .1
    # plt.imshow(mask)
    # plt.show()

    mask = scipy.ndimage.binary_fill_holes(mask)

    labeled = skimage.morphology.label(mask)

    props = skimage.measure.regionprops(labeled)
    count = 0
    for p in props:
        if p.area < area:
            labeled[labeled==p.label] = 0
            count += 1
    return labeled

def get_feature_df(labeled, img, channels, channel_info):
    annots = sorted(channel_info.keys())
    data = []
    columns =  channels + annots
    props = skimage.measure.regionprops(labeled, intensity_image=rearrange(img, 'c h w -> h w c'))
    for p in props:
        row = list(p.intensity_mean)
        r1, c1, r2, c2 = p.bbox
        for annot in annots:
            tile = annot_to_mask[annot][r1:r2, c1:c2]
            frac = np.count_nonzero(tile[p.image] > 0) / np.count_nonzero(p.image)
            row.append(frac)
        data.append(row)
    return pd.DataFrame(data=data, columns=columns, index=[p.label for p in props])

def get_feature_df_vishd(labeled, adata):
    data = []
    columns =  adata.var.index.to_list()
    ys, xs = adata.obs['y_location'].astype(int).to_list(), adata.obs['x_location'].astype(int).to_list()
    adata.obs['region'] = labeled[ys, xs]
    
    pool = [x for x in np.unique(labeled) if x]
    for l in pool:
        f = adata[adata.obs['region']==l]
        if f.shape[0]:
            means = np.asarray(f.X.mean(0))[0]
        else:
            means = [np.nan] * adata.shape[1]
        data.append(means)
    
    return pd.DataFrame(data=data, columns=columns, index=pool)

In [13]:
case_to_channel_info = {
    'HT206B1': {
        'xenium_0': {
            'epithelial': ('EPCAM', 5),
            'myoepithelial': ('ACTA2', 5),
        },
        'xenium_1': {
            'epithelial': ('EPCAM', 5),
            'myoepithelial': ('ACTA2', 5),
        },
        'multiplex_0': {
            'epithelial': ('pan-cytokeratin', 40),
            'myoepithelial': ('sma', 40),
        },
        'multiplex_1': {
            'epithelial': ('pan-cytokeratin', 40),
            'myoepithelial': ('sma', 40),
        }
    },
    'HT704B1': {
        'xenium_0': {
            'epithelial': ('EPCAM', 5),
            'myoepithelial': ('KRT5', 5),
        },
        'cosmx_0': {
            'epithelial': ('EPCAM', 5),
            'myoepithelial': ('KRT5', 5),
        },
        'multiplex_0': {
            'epithelial': ('pan-cytokeratin', 40),
            'myoepithelial': ('ck5', 40),
        },
        'vishd_0': {
            'epithelial': ('EPCAM', 15),
            'myoepithelial': ('KRT5', 5),
        },
        'vishd_multpred': {
            'epithelial': ('pan-cytokeratin', 60),
        }
    },
    'HT397B1': {
        'multiplex_0': {
            'epithelial': ('pan-cytokeratin', 40),
            'myoepithelial': ('ck14', 20),
        },
        'multiplex_1': {
            'epithelial': ('pan-cytokeratin', 40),
            'myoepithelial': ('ck14', 20),
        },
        'multiplex_2': {
            'epithelial': ('pan-cytokeratin', 40),
            'myoepithelial': ('ck14', 20),
        }
    },
    'HT339B2-H1': {
        'multiplex_0': {
            'epithelial': ('pan-cytokeratin', 5),
            'myoepithelial': ('ck5', 40),
        }
    },
    'HT565B1-H2': {
        'multiplex_0': {
            'epithelial': ('pan-cytokeratin', 40),
            'myoepithelial': ('ck5', 40),
        }
    },
    'HT891Z1': {
        'xenium_0': {
            'epithelial': ('EPCAM', 15),
            'myoepithelial': ('KRT5', 5),
        },
        'multiplex_0': {
            'epithelial': ('e-cadherin', 55),
            'myoepithelial': ('ck5', 40),
        },
        'vishd_0': {
            'epithelial': ('EPCAM', 15),
            'myoepithelial': ('KRT5', 5),
        },
        'vishd_multpred': {
            'epithelial': ('e-cadherin', 60),
        }
    },
    'HT913Z1': {
        'xenium_0': {
            'epithelial': ('EPCAM', 15),
            'myoepithelial': ('TP63', 5),
        },
        'xenium_1': {
            'epithelial': ('EPCAM', 14),
            'myoepithelial': ('KRT5', 5),
        },
        'multiplex_0': {
            'epithelial': ('e-cadherin', 100),
            'myoepithelial': ('ck5', 40),
        }
    },
    'S18-5591-C8': {
        'xenium_0': {
            'epithelial': ('EPCAM', 5),
            'myoepithelial': ('KRT5', 5),
        },
        'multiplex_0': {
            'epithelial': ('e-cadherin', 3),
            'myoepithelial': ('ck5', 40),
        }
    },
    'S18-9906': {
        'xenium_0': {
            'epithelial': ('EPCAM', 15),
            'myoepithelial': ('KRT7', 5),
        },
        'multiplex_0': {
            'epithelial': ('e-cadherin', 30),
            'myoepithelial': ('ck5', 40),
        }
    },
    
        
}

In [14]:
import torch


In [15]:
for case in case_order:
    print(case)
    imaris_dir = project_dir / case / 'imaris' / 'rois'
    config = yaml.safe_load(open(project_dir / case / 'registered' / 'metadata.yaml'))
    config = alter_filesystem(config, source_root, target_root)
    
    meta = json.load(open(imaris_dir / 'tiled' / 'metadata.json'))
    
    sid_to_channels = meta['sid_to_channels']
    dtype_ident_to_dtype = meta['dtype_ident_to_dtype']
    dtype_ident_to_channels = meta['dtype_ident_to_channels']
    sid_to_dtype_ident = meta['sid_to_dtype_ident']
    tiling_size = meta['tiling_size']
    size = meta['size']
    fullres_size = meta['fullres_size']
    dtype_ident_to_tiled_fps = meta['dtype_ident_to_tiled_fps']
    
    dtype_ident_to_tiled = {dti:tifffile.imread(fp) for dti, fp in dtype_ident_to_tiled_fps.items()}
    for dti, tiled in dtype_ident_to_tiled.items():
        print(dti, tiled.shape)
    
    sid_to_tiled_idx = {}
    for dtype_ident in dtype_ident_to_channels.keys():
        if 'vis' not in dtype_ident:
            dtype = dtype_ident_to_dtype[dtype_ident]
            sids = [entry['sid'] for entry in config['sections'] if sid_to_dtype_ident[entry['sid']]==dtype_ident]
            for i, sid in enumerate(sids):
                sid_to_tiled_idx[sid] = i

    meta_ct = json.load(open(imaris_dir / 'tiled_cell_types' / 'metadata.json'))

    sid_to_channels_ct = meta_ct['sid_to_channels']
    dtype_ident_to_dtype_ct = meta_ct['dtype_ident_to_dtype']
    dtype_ident_to_channels_ct = meta_ct['dtype_ident_to_channels']
    sid_to_dtype_ident_ct = meta_ct['sid_to_dtype_ident']
    dtype_ident_to_tiled_fps_ct = meta_ct['dtype_ident_to_tiled_fps']
    
    dtype_ident_to_tiled_ct = {dti:tifffile.imread(fp) for dti, fp in dtype_ident_to_tiled_fps_ct.items()}
    for dti, tiled in dtype_ident_to_tiled_ct.items():
        print(dti, tiled.shape)
    
    sid_to_tiled_idx_ct = {}
    for dtype_ident in dtype_ident_to_channels_ct.keys():
        dtype = dtype_ident_to_dtype_ct[dtype_ident]
        sids = [entry['sid'] for entry in config['sections'] if sid_to_dtype_ident.get(entry['sid'])==dtype_ident]
        for i, sid in enumerate(sids):
            sid_to_tiled_idx_ct[sid] = i

    data = pickle.load(
        open(project_dir / case / 'mushroom_50res' / f'outputs.pkl', 'rb')
    )
    mushroom_config = yaml.safe_load(
        open(project_dir / case / 'mushroom_50res' / f'config.yaml', 'rb')
    )
    mushroom_res = mushroom_config['trainer_kwargs']['target_resolution']

    for entry in config['sections']:
        sid = entry['sid']
        position = entry['position']
        dti = sid_to_dtype_ident[sid]
        dtype = dti.split('_')[0]
        channels = dtype_ident_to_channels[dti]
        print(sid, dti, len(channels))

        tups = [x for x in data['section_ids'] if x[1] == dtype]
        if (sid, dtype) not in tups:
            continue
        idx = tups.index((sid, dtype))
        nbhds = data['dtype_to_clusters'][dtype][-1][idx]
        nbhds = utils.rescale(nbhds, size=size, dim_order='h w', target_dtype=nbhds.dtype, antialias=False, interpolation=TF.InterpolationMode.NEAREST)
        nbhd_hots = rearrange(torch.nn.functional.one_hot(torch.tensor(nbhds)).numpy(), 'h w c -> c h w')
        nbhd_order = [f'nbhd{x}' for x in np.arange(nbhds.max() + 1)]

        if dtype == 'multiplex':
            channels = [alter_name(x) for x in channels]

        if dti in dtype_ident_to_tiled and 'he_' not in dti and 'he_' not in dti:
            img = dtype_ident_to_tiled[dti][sid_to_tiled_idx[sid]]
            channel, cut = case_to_channel_info[case][dti]['epithelial']

            labeled = generate_labeled_regions(img[channels.index(channel)], cut)
            channel_info = case_to_channel_info[case][dti]

            expanded = skimage.segmentation.expand_labels(labeled, distance=2)
            m = (expanded > 0) & ~(labeled > 0)
            ring = expanded.copy()
            ring[~m] = 0
            
            expanded = skimage.segmentation.expand_labels(labeled, distance=10)
            m = (expanded > 0) & ~(skimage.segmentation.expand_labels(labeled, distance=2) > 0)
            tme = expanded.copy()
            tme[~m] = 0
            
            annot_to_mask = {
                k: img[channels.index(info[0])] > info[1] for k, info in channel_info.items()
            }
            
            stack = []
            for name, x in zip(['region', 'boundary', 'tme'], [labeled, ring, tme]):
                df = get_feature_df(x, img, channels, channel_info)
                df.columns = [f'{name}_{c}' for c in df.columns]
                stack.append(df)

                if sid in sid_to_tiled_idx_ct:
                    ct_dti = sid_to_dtype_ident_ct[sid]
                    ct_img = dtype_ident_to_tiled_ct[ct_dti][sid_to_tiled_idx_ct[sid]]
                    ct_channels = dtype_ident_to_channels_ct[dti]
                    df = get_feature_df(x, ct_img, ct_channels, {})
                    df.columns = [f'{name}_celltypes_{c}' for c in df.columns]
                    stack.append(df)

                df = get_feature_df(x, nbhd_hots, nbhd_order, {})
                df.columns = [f'{name}_neighborhoods_{c}' for c in df.columns]
                stack.append(df)

            df = pd.concat(stack, axis=1)
        elif dti == 'vishd_0':
            fp = entry['data'][0]['filepath']
            adata = sc.read_h5ad(fp)
            adata.var_names_make_unique()
            adata.obs['y_location'] = adata.obs['y_location'] // 10
            adata.obs['x_location'] = adata.obs['x_location'] // 10
            pred_fp = project_dir / f'{case}' / 'multiplex_prediction' / 'predictions_vishd' / f'{sid}.ome.tif'
            pred_channels, data = multiplex.extract_ome_tiff(pred_fp, as_dict=False)
            pred_channels = [alter_name(x) for x in pred_channels]
            channel, cut = case_to_channel_info[case]['vishd_multpred']['epithelial']
            img = data[pred_channels.index(channel)]
            img = utils.rescale(img, size=size, dim_order='h w', target_dtype=img.dtype)
            
            labeled = generate_labeled_regions(img, cut)

            channel_info = case_to_channel_info[case][dti]
            expanded = skimage.segmentation.expand_labels(labeled, distance=2)
            m = (expanded > 0) & ~(labeled > 0)
            ring = expanded.copy()
            ring[~m] = 0
            
            expanded = skimage.segmentation.expand_labels(labeled, distance=10)
            m = (expanded > 0) & ~(skimage.segmentation.expand_labels(labeled, distance=2) > 0)
            tme = expanded.copy()
            tme[~m] = 0

            stack = []
            for name, x in zip(['region', 'boundary', 'tme'], [labeled, ring, tme]):
                df = get_feature_df_vishd(x, adata)
                df.columns = [f'{name}_{c}' for c in df.columns]
                stack.append(df)

                if sid in sid_to_tiled_idx_ct:
                    ct_dti = sid_to_dtype_ident_ct[sid]
                    ct_img = dtype_ident_to_tiled_ct[ct_dti][sid_to_tiled_idx_ct[sid]]
                    ct_channels = dtype_ident_to_channels_ct[dti]
                    df = get_feature_df(x, ct_img, ct_channels, {})
                    df.columns = [f'{name}_celltypes_{c}' for c in df.columns]
                    stack.append(df)

                df = get_feature_df(x, nbhd_hots, nbhd_order, {})
                df.columns = [f'{name}_neighborhoods_{c}' for c in df.columns]
                stack.append(df)
                
            df = pd.concat(stack, axis=1)

        else:
            labeled, ring, tme, df = None, None, None, None

        if labeled is not None:
            tifffile.imwrite(output_dir / f'{sid}_regions.tif', labeled, compression='LZW')
            tifffile.imwrite(output_dir / f'{sid}_boundaries.tif', ring, compression='LZW')
            tifffile.imwrite(output_dir / f'{sid}_tme.tif', tme, compression='LZW')
                
            df.to_csv(output_dir / f'{sid}_regions.txt.gz', sep='\t')
        

HT206B1
xenium_0 (3, 477, 653, 605)
he_0 (3, 3, 653, 605)
multiplex_0 (3, 37, 653, 605)
multiplex_1 (3, 38, 653, 605)
xenium_1 (3, 377, 653, 605)
xenium_0 (6, 14, 653, 605)
multiplex_0 (6, 6, 653, 605)
HT206B1-U1 xenium_0 477
HT206B1-U4 he_0 3
HT206B1-U2 multiplex_0 37
HT206B1-U5 multiplex_1 38
HT206B1-U8 xenium_1 377
HT206B1-U9 xenium_0 477
HT206B1-U10 multiplex_0 37
HT206B1-U12 he_0 3
HT206B1-U13 multiplex_1 38
HT206B1-U16 xenium_1 377
HT206B1-U17 xenium_0 477
HT206B1-U18 multiplex_0 37
HT206B1-U20 he_0 3
HT206B1-U21 multiplex_1 38
HT206B1-U24 xenium_1 377
HT268B1
visium_0 (4, 14, 677, 669)
HT268B1-U2 visium_0 36601
HT268B1-U12 visium_0 36601
HT268B1-U22 visium_0 36601
HT268B1-U32 visium_0 36601
HT339B2-H1
multiplex_0 (1, 33, 678, 668)
visium_0 (1, 11, 678, 668)
multiplex_0 (1, 6, 678, 668)
HT339B2-H1-U1 visium_0 17943
HT339B2-H1-U2 multiplex_0 33
HT397B1
multiplex_0 (2, 25, 678, 669)
batch2_he_0 (6, 3, 678, 669)
multiplex_1 (1, 27, 678, 669)
multiplex_2 (1, 25, 678, 669)
visium_0 (2

In [17]:
output_dir

PosixPath('/data/estorrs/mushroom/data/projects/submission_v1/analysis/region_characterization')